In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
import scipy.signal as signal
import os
%matplotlib inline

In [ ]:
input_folder = "./radiative_loss_results/"
output_folder = "./radiative_loss_figures/"
try:
    os.mkdir(output_folder)
except FileExistsError:
    pass

font_scale = 1.75
figsize = (8, 6)
double_figsize = (8, 10)
dpi = 300
handlelength = .8
markersize=10
def plot_style():
    sns.set(style='ticks', font_scale=font_scale)
    matplotlib.rcParams['mathtext.fontset'] = 'stix'
    matplotlib.rcParams['font.family'] = 'STIXGeneral'
    matplotlib.rcParams['legend.fontsize'] = 17.5
    matplotlib.rcParams["legend.frameon"] = False
    
pfilter_label = 'Purcell filter'
tline_label = 'Transmission line'
labels = [tline_label, pfilter_label]
colors_2 = ['red', 'blue']
colors_3 = ['indigo', 'darkcyan', 'coral']

def parse_complex(s):
    if " + " in s:
        r, i = s.split(" + ")
        return float(r) + 1j * float(i.split("im")[0])
    elif " - " in s:
        r, i = s.split(" - ")
        return float(r) - 1j * float(i.split("im")[0])

# Scattering

In [ ]:
df = pandas.read_csv(input_folder + "S12.csv")
freqs = np.array(df['freqs'])
tline_S = np.array([parse_complex(s) for s in df['tline_S']])
pfilter_S = np.array([parse_complex(s) for s in df['pfilter_S']])

In [ ]:
plot_style()
fig, ax1 = plt.subplots(1, figsize=figsize)
s12s = [tline_S, pfilter_S]
for l, s12, c, style in zip(labels, s12s, colors_2, ['-', '--']):
    plt.plot(freqs/1e9, np.abs(s12), style, label=l, color=c)
ax1.legend(handlelength=handlelength, bbox_to_anchor=(.65,.2))
ax1.set_xlabel('Frequency [GHz]')
ax1.set_ylabel(u'$|S_{21}|$')

left, bottom, width, height = [0.7, 0.25, 0.2, 0.2]
ax2 = fig.add_axes([left, bottom, width, height])
for l, s12, c, style in zip(labels, s12s, colors_2, ['-', '--']):
    ax2.plot(freqs/1e9, np.abs(s12), style, label=l, color=c)
ax2.set_xlim([5.9, 5.9075])
plt.savefig(output_folder + 'mag_s12.png', dpi=dpi)

# Effective capacitance

In [ ]:
df = pandas.read_csv(input_folder + "Y.csv")
freqs = np.array(df['freqs'])
tline_Y = np.array([parse_complex(s) for s in df['tline_Y']])
pfilter_Y = np.array([parse_complex(s) for s in df['pfilter_Y']])

In [ ]:
plot_style()
fig = plt.figure(figsize=figsize)
plt.plot(freqs/1e9, np.imag(tline_Y) * 1e3,
         label=u'Transmission line $Y_{e}$', color=colors_2[0])
plt.plot(freqs/1e9, np.imag(pfilter_Y) * 1e3, '--',
         label=u'Purcell filter $Y_{e}$', color=colors_2[1])
cg = 7e-15 # qubit - resonator coupling capacitance
plt.plot(freqs/1e9, cg * (2 * np.pi * freqs) * 1e3,
         label=u'$C_{c} s$', color=colors_3[1])
plt.xlabel(u'$\omega_q^{0}/2\pi$ [GHz]')
plt.ylabel(u'Im$[Y]$  $[10^{-3} \hspace{0.5} \Omega^{-1}$]')
plt.ylim([-0.1, 0.5])
plt.xlim([0, 10])
plt.legend(loc=2, handlelength=handlelength)
plt.tight_layout()
plt.savefig(output_folder + 'c_env_fit.png', dpi=dpi)

# Eigenmodes

In [ ]:
df = pandas.read_csv(input_folder + "tline_eigenmodes.csv")
bare_qubit_freqs = np.array(df["freqs"])
tline_results = {"Transmon": np.array([parse_complex(s)/(2 * np.pi) for s in df["x1"]]),
                 "Resonator mode 1": np.array([parse_complex(s)/(2 * np.pi) for s in df["x2"]]),
                 "Resonator mode 2": np.array([parse_complex(s)/(2 * np.pi) for s in df["x3"]])}
tline_admittance_T1 = df["T1_Y"]
df = pandas.read_csv(input_folder + "pfilter_eigenmodes.csv")
pfilter_results = {"Transmon": np.array([parse_complex(s)/(2 * np.pi) for s in df["x1"]]),
                   "Resonator mode 1": np.array([parse_complex(s)/(2 * np.pi) for s in df["x2"]]),
                   "Resonator mode 2": np.array([parse_complex(s)/(2 * np.pi) for s in df["x3"]])}
pfilter_admittance_T1 = df["T1_Y"]

In [ ]:
tline_rel_mins = signal.argrelmin(np.array(tline_results["Transmon"]).imag-bare_qubit_freqs)[0]+1
pfilter_rel_mins = signal.argrelmin(np.array(pfilter_results["Transmon"]).imag-bare_qubit_freqs)[0]+1
def plot_data(func, data, rel_mins, color, label, linestyle):
    data = np.array(data)
    func(bare_qubit_freqs[:rel_mins[0]]/1e9, data[:rel_mins[0]], linestyle, label=label, c=color)
    func(bare_qubit_freqs[rel_mins[0]:rel_mins[1]]/1e9, data[rel_mins[0]:rel_mins[1]], linestyle, c=color)
    func(bare_qubit_freqs[rel_mins[1]:]/1e9, data[rel_mins[1]:], linestyle, c=color)
def diag_T1(data):
    ans = 1/(-2 * 2 * np.pi * np.array(data).real)
    return ans
def diag_freq(data):
    return np.array(data).imag

In [ ]:
plot_style()
dashes = (2.55,2)
fig, axs = plt.subplots(2, 2, figsize=double_figsize)

# PLOT 00
func = axs[0,0].plot
plot_data(func, diag_freq(tline_results["Transmon"])/1e9, tline_rel_mins, colors_3[0],
          "Transmon", '-')
plot_data(func, diag_freq(tline_results["Resonator mode 1"])/1e9, tline_rel_mins, colors_3[1],
          "Resonator mode 1", '-')
plot_data(func, diag_freq(tline_results["Resonator mode 2"])/1e9, tline_rel_mins, colors_3[2],
          "Resonator mode 2", '-')
# INSET
left, bottom, width, height = [0.421, 0.68, 0.125, 0.125]
ax_0 = fig.add_axes([left, bottom, width, height])
func = ax_0.plot
plot_data(func, diag_freq(tline_results["Transmon"])/1e9, tline_rel_mins, colors_3[0],
          "Transmon", '-')
plot_data(func, diag_freq(tline_results["Resonator mode 1"])/1e9, tline_rel_mins, colors_3[1],
          "Resonator mode 1", '-')
ax_0.set_xlim([5, 6.5])
ax_0.set_ylim([5.5,6.3])

# PLOT 10
func = axs[1,0].semilogy
plot_data(func, diag_T1(tline_results["Transmon"]), tline_rel_mins, colors_3[0],
          '', '-')
plot_data(func, diag_T1(tline_results["Resonator mode 1"]), tline_rel_mins, colors_3[1],
          '', '-')
plot_data(func, diag_T1(tline_results["Resonator mode 2"]), tline_rel_mins, colors_3[2],
          '', '-')
def _func(*args, **kwargs):
    kwargs['dashes']=dashes
    func(*args,**kwargs)
plot_data(_func, tline_admittance_T1, tline_rel_mins,
          "lightcoral", u'$\\frac{C}{Re[Y]}$', '--')
# INSET
left, bottom, width, height = [0.421, 0.37, 0.125, 0.125]
ax_1 = fig.add_axes([left, bottom, width, height])
func = ax_1.plot
plot_data(func, diag_T1(tline_results["Transmon"]), tline_rel_mins, colors_3[0],
          '', '-')
plot_data(func, diag_T1(tline_results["Resonator mode 1"]), tline_rel_mins, colors_3[1],
          '', '-')
def _func(*args, **kwargs):
    kwargs['dashes']=dashes
    func(*args,**kwargs)
plot_data(_func, tline_admittance_T1, tline_rel_mins,
          "lightcoral", u'$\\frac{C}{Re[Y]}$', '--')
ax_1.set_xlim([5, 6.5])
ax_1.set_ylim([1e-8,10**(-6.2)])

# PLOT 01
func = axs[0,1].plot
plot_data(func, diag_freq(pfilter_results["Transmon"])/1e9, pfilter_rel_mins, colors_3[0],
          '', '-')
plot_data(func, diag_freq(pfilter_results["Resonator mode 1"])/1e9, pfilter_rel_mins, colors_3[1],
          '', '-')
plot_data(func, diag_freq(pfilter_results["Resonator mode 2"])/1e9, pfilter_rel_mins, colors_3[2],
          '', '-')
# INSET
left, bottom, width, height = [0.864, 0.68, 0.125, 0.125]
ax_2 = fig.add_axes([left, bottom, width, height])
func = ax_2.plot
plot_data(func, diag_freq(pfilter_results["Transmon"])/1e9, pfilter_rel_mins, colors_3[0],
          '', '-')
plot_data(func, diag_freq(pfilter_results["Resonator mode 1"])/1e9, pfilter_rel_mins, colors_3[1],
          '', '-')
ax_2.set_xlim([5, 6.5])
ax_2.set_ylim([5.5,6.3])

# PLOT 11
func = axs[1,1].semilogy
plot_data(func, diag_T1(pfilter_results["Transmon"]), pfilter_rel_mins, colors_3[0],
          '', '-')
plot_data(func, diag_T1(pfilter_results["Resonator mode 1"]), pfilter_rel_mins, colors_3[1],
          '', '-')
plot_data(func, diag_T1(pfilter_results["Resonator mode 2"]), pfilter_rel_mins, colors_3[2],
          '', '-')
def _func(*args, **kwargs):
    kwargs['dashes']=dashes
    func(*args,**kwargs)
plot_data(_func, pfilter_admittance_T1, pfilter_rel_mins,
          "lightcoral", '', '--')
# INSET
left, bottom, width, height = [0.864, 0.37, 0.125, 0.125]
ax_3 = fig.add_axes([left, bottom, width, height])
func = ax_3.plot
plot_data(func, diag_T1(pfilter_results["Transmon"]), pfilter_rel_mins, colors_3[0],
          '', '-')
plot_data(func, diag_T1(pfilter_results["Resonator mode 1"]), pfilter_rel_mins, colors_3[1],
          '', '-')
def _func(*args, **kwargs):
    kwargs['dashes']=dashes
    func(*args,**kwargs)
plot_data(_func, pfilter_admittance_T1, pfilter_rel_mins,
          "lightcoral", '', '--')
ax_3.set_xlim([5, 6.5])
ax_3.set_ylim([1e-8,10**(-6.2)])


axs[0,0].set_xticks([])
axs[0,1].set_xticks([])
axs[0,1].set_yticks([])
axs[1,1].set_yticks([])
axs[1,0].set_ylim([10**(-9.1), 7])
axs[1,1].set_ylim([10**(-9.1), 7])
axs[0,0].set_ylabel('Frequency [GHz]')
axs[1,0].set_ylabel(u'$T_1$ [s]')
axs[1,0].set_xlabel(u'$\omega_q^{0}/2\pi$ [GHz]')
axs[1,1].set_xlabel(u'$\omega_q^{0}/2\pi$ [GHz]')
axs[0,0].legend(loc=2, handlelength=handlelength)
axs[1,0].legend(loc=2, handlelength=.75)
axs[0,0].set_title(tline_label)
axs[0,1].set_title(pfilter_label)
plt.tight_layout(pad=0.0)
plt.savefig(output_folder + 'complex_frequencies.png', dpi=dpi)

# Convergence

In [ ]:
tline_eigenvalues = np.load(input_folder + "tline_eigenvalues.npy")
pfilter_eigenvalues = np.load(input_folder + "pfilter_eigenvalues.npy")
mode_names = ["Transmon", "Resonator mode 1", "Resonator mode 2"]

In [ ]:
plot_style()
fig, axs = plt.subplots(2, 2, figsize=double_figsize)

def make_plot(ax, f, results_object, y_min, y_max, do_axhline=True, label_flag=True):
    for i, mode_name in enumerate(mode_names):
        a=f(results_object[i, :, -3])
        b=f(results_object[i, :, -2])
        c=f(results_object[i, :, -1])
        y1 = np.abs((a-c)/c)
        y2 = np.abs((b-c)/c)
        y1_label = mode_name + u' $\delta_2$' if (not label_flag) else None
        y2_label = mode_name + " $\delta_1$" if label_flag else None
        ax.semilogy(bare_qubit_freqs/1e9, y1, "--", markersize=3,
                    label=y1_label, color=colors_3[i])
        ax.semilogy(bare_qubit_freqs/1e9, y2, markersize=3,
                    label=y2_label, color=colors_3[i])
        if do_axhline:
            ax.axhline(1e-2, linestyle='--', c='black')
        ax.set_ylim([y_min, y_max])
        ax.minorticks_off()

make_plot(axs[0,0], diag_freq, tline_eigenvalues, 1e-10, 2e-4, do_axhline=False, label_flag=True)
make_plot(axs[1,0], diag_T1, tline_eigenvalues, 1e-7, 2e1)
make_plot(axs[0,1], diag_freq, pfilter_eigenvalues, 1e-10, 2e-4, do_axhline=False, label_flag=False)
make_plot(axs[1,1], diag_T1, pfilter_eigenvalues, 1e-7, 2e1)

axs[0,0].set_xticks([])
axs[0,1].set_xticks([])
axs[0,1].set_yticks([])
axs[1,1].set_yticks([])

axs[0,0].legend(loc=4, handlelength=handlelength)
axs[0,1].legend(loc=4, handlelength=handlelength)

axs[0,0].set_ylabel('Frequency rel. diff.')
axs[1,0].set_ylabel(u'$T_1$ rel. diff.')
axs[1,0].set_xlabel(u'$\omega_q^{0}/2\pi$ [GHz]')
axs[1,1].set_xlabel(u'$\omega_q^{0}/2\pi$ [GHz]')
axs[0,0].set_title(tline_label)
axs[0,1].set_title(pfilter_label)
plt.tight_layout(pad=0.0)
plt.savefig(output_folder + 'tline_pfilter_convergence.png', dpi=dpi)